In [1]:
import json
import numpy as np
import panel as pn
import pandas as pd
from statsmodels.regression.linear_model import OLSResults
from typing import Dict
import param
pn.extension()

In [2]:
# load model
model = OLSResults.load("app-code-model.pickle")
zooms = range(6)

In [3]:
def predict_tile_times(
       lat_dim_size: int,
       lon_dim_size: int,
       lat_chunk_size: int,
       lon_chunk_size: int,
       time_chunk_size: int,
       data_type: str
   ) -> Dict[str, float]:
    number_of_spatial_chunks = (lat_dim_size/lat_chunk_size) * (lon_dim_size/lon_chunk_size)
    data_dtype = np.dtype(data_type)
    chunk_size_mb = lat_chunk_size * lon_chunk_size * time_chunk_size * data_dtype.itemsize/1024/1024
    values_to_predict = {
        'const': [1],
        'number_of_spatial_chunks': [number_of_spatial_chunks],
        'chunk_size_mb': [chunk_size_mb]
    }
    tile_times = {}
    for zoom in zooms:
        zooms_to_predict = {f'zoom {zoom}': [0] for zoom in zooms}        
        zooms_to_predict[f'zoom {zoom}'] = [1]
        values_to_predict.update(zooms_to_predict)
        prediction = model.predict(pd.DataFrame(values_to_predict))
        tile_times[f'zoom {zoom}'] = prediction
    return pd.DataFrame(tile_times).transpose()

In [4]:
# list all widgets
lat_dim_size_widget = pn.widgets.IntInput(name='Latitidude Dimension Size', value=100, step=100, start=0)
lon_dim_size_widget = pn.widgets.IntInput(name='Longitude Dimension Size', value=100, step=100, start=0)
lat_chunk_size_widget = pn.widgets.IntInput(name='Latitude Chunk Size', value=100, step=100, start=0)
lon_chunk_size_widget = pn.widgets.IntInput(name='Longitidude Chunk Size', value=100, step=100, start=0)
time_chunk_size_widget = pn.widgets.IntInput(name='Time Chunk Size', value=1, step=1, start=0)
data_type_widget = pn.widgets.Select(name="Data Type", value="float16", options=["float16", "float32", "float64"])

In [5]:
bound_results = pn.bind(
    predict_tile_times,
    lat_dim_size=lat_dim_size_widget,
    lon_dim_size=lon_dim_size_widget,
    lat_chunk_size=lat_chunk_size_widget,
    lon_chunk_size=lon_chunk_size_widget,
    time_chunk_size=time_chunk_size_widget,
    data_type = data_type_widget
)

In [6]:
first_app = pn.Column(
    lat_dim_size_widget,
    lon_dim_size_widget,
    lat_chunk_size_widget,
    lon_chunk_size_widget,
    time_chunk_size_widget,
    data_type_widget,
    bound_results)

first_app

Column
    [0] IntInput(name='Latitidude Dimension S..., start=0, step=100, value=100)
    [1] IntInput(name='Longitude Dimension S..., start=0, step=100, value=100)
    [2] IntInput(name='Latitude Chunk Size', start=0, step=100, value=100)
    [3] IntInput(name='Longitidude Chunk Size', start=0, step=100, value=100)
    [4] IntInput(name='Time Chunk Size', start=0, value=1)
    [5] Select(name='Data Type', options=['float16', 'float32', ...], value='float16')
    [6] ParamFunction(function, _pane=DataFrame, defer_load=False)